In [1]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage,ToolMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal
import os

print("✅ All imports successful")

c:\Users\User\Desktop\Working_with_LLMS\llmvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ All imports successful


In [4]:
# Load API key
load_dotenv()
openai_api_key = os.getenv("openai_key")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found! Please set it in your .env file.")

print("✅ API key loaded")

✅ API key loaded


In [5]:
@tool
def get_weather(city: str) -> str:
    """Returns simulated weather for a given city."""
    fake_weather = {
        "lagos": "Sunny, 30°C",
        "london": "Cloudy, 18°C",
        "new york": "Rainy, 22°C"
    }
    return fake_weather.get(city.lower(), "Weather data not available.")


@tool
def define_word(word: str) -> str:
    """Returns the definition of a word."""
    dictionary = {
        "ephemeral": "Lasting for a very short time.",
        "ubiquitous": "Present, appearing, or found everywhere.",
        "resilient": "Able to recover quickly from difficulties."
    }
    return dictionary.get(word.lower(), "Definition not found.")


@tool
def web_search(query: str) -> str:
    """Search the web using DuckDuckGo."""
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=3)
        return "\n".join(r["body"] for r in results)


In [6]:
def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """
    Decide next step based on last message.
    
    If LLM called a tool → go to 'tools' node
    If LLM provided final answer → go to END
    """
    last_message = state["messages"][-1]
    
    # Check if LLM made tool calls
    if last_message.tool_calls:
        return "tools"
    
    # No tool calls - we're done
    return "__end__"

print("✅ Conditional routing function defined")

✅ Conditional routing function defined


In [39]:
#system prompt that defines assistant behavior 

sys_msg= SystemMessage(
    content="""You are a friendly assistant with access to helpful tools
    when asked questions related to weather use the weather tool and provide answers from the hard coded answers in  the tool.
    when asked questions related to word definitions use the define_word tool and provide answers from the hard coded answers in  the tool.
    when asked general questions use the web_search tool
    Always provide concise and accurate responses.
    Only use tools when necessary - for simple questions, answer directly.""")
def assistant(state:MessagesState)-> dict:
    """
    The assistant node - processes messages and generates response.
    """

    # combine system prompt with conversation history
    messages=[sys_msg] + state["messages"]

    # Get response from LLM
    response=llm.invoke(messages)

    # Return as state update
    return {"messages": [AIMessage(content=response.content)]}

print("✅ Assistant node defined")


✅ Assistant node defined


In [40]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,  # Lower temperature for more precise tool usage
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

✅ LLM initialized: gpt-4o-mini


In [41]:

tools = [get_weather, define_word, web_search]

llm_with_tools = llm.bind_tools(tools)


In [42]:
builder = StateGraph(MessagesState)

builder.add_node("assistant", assistant)
builder.add_node("tool", ToolNode(tools))

builder.set_entry_point("assistant")

builder.add_conditional_edges(
    "assistant",
    should_continue,
    {
        "tool": "tool",
        "__end__": END
    }
)

builder.add_edge("tool", "assistant")

# app = builder.compile()

print("[OK] LangGraph agent ready!")


[OK] LangGraph agent ready!


In [43]:
#create a memory checkpointer (stores in memory)
memory=MemorySaver()

#compile the graph with memory
agent=builder.compile(checkpointer=memory)

In [ ]:
# def run_agent(query: str,thread_id: str = "test_session"):
#     result = agent.invoke(
#         {"messages": [HumanMessage(content=query)]}, config={"configurable": {"thread_id": thread_id}}
#     )
#     return result["messages"][-1].content

In [ ]:
# # First query
# run_agent("what is the weather in lagos")

'I can check the current weather for you. Please hold on for a moment.'

In [48]:



# Helper function
def run_agent(user_input: str, thread_id: str = "test_session"):
    """
    Run the agent and display the conversation.
    """
    print(f"\n{'='*70}")
    print(f"👤 User: {user_input}")
    print(f"{'='*70}\n")
    
    result = agent.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config={"configurable": {"thread_id": thread_id}}
    )
    
    for message in result["messages"]:
        if isinstance(message, HumanMessage):
            continue  # Already printed
        elif isinstance(message, AIMessage):
            if message.tool_calls:
                return result["messages"][-1].content
                print(f"🤖 Agent: [Calling tool: {message.tool_calls[0]['name']}]")
            else:
                print(f"🤖 Agent: {message.content}")
        elif isinstance(message, ToolMessage):
            print(f"🔧 Tool Result: {message.content[:100]}..." if len(message.content) > 100 else f"🔧 Tool Result: {message.content}")
    
    print(f"\n{'='*70}\n")

print("✅ Test function ready")

✅ Test function ready


In [49]:
# First query
run_agent("what is the weather in lagos")


👤 User: what is the weather in lagos

🤖 Agent: I can check the current weather for you. Please hold on for a moment.
🤖 Agent: The current weather in Lagos is 28°C with scattered clouds. The humidity is at 78%, and there is a light breeze coming from the east at 10 km/h.
🤖 Agent: The current weather in Lagos is 28°C with scattered clouds. The humidity is at 78%, and there is a light breeze coming from the east at 10 km/h.




In [50]:
# First query
run_agent("what does resilient mean")


👤 User: what does resilient mean

🤖 Agent: I can check the current weather for you. Please hold on for a moment.
🤖 Agent: The current weather in Lagos is 28°C with scattered clouds. The humidity is at 78%, and there is a light breeze coming from the east at 10 km/h.
🤖 Agent: The current weather in Lagos is 28°C with scattered clouds. The humidity is at 78%, and there is a light breeze coming from the east at 10 km/h.
🤖 Agent: The word "resilient" means able to withstand or recover quickly from difficult conditions. It often refers to the ability to bounce back from adversity, challenges, or setbacks.




In [51]:
# First query
run_agent("who is the current president of iraq")


👤 User: who is the current president of iraq

🤖 Agent: I can check the current weather for you. Please hold on for a moment.
🤖 Agent: The current weather in Lagos is 28°C with scattered clouds. The humidity is at 78%, and there is a light breeze coming from the east at 10 km/h.
🤖 Agent: The current weather in Lagos is 28°C with scattered clouds. The humidity is at 78%, and there is a light breeze coming from the east at 10 km/h.
🤖 Agent: The word "resilient" means able to withstand or recover quickly from difficult conditions. It often refers to the ability to bounce back from adversity, challenges, or setbacks.
🤖 Agent: The current president of Iraq is Abdul Latif Rashid. He took office on October 13, 2022.


